# Scoring benchmark


## Params

In [1]:
MUTATIONS = [0.8, 0.6, 0.4, 0.2, 0.1, 0.05, 0.01]
REMOVALS = [0.8, 0.6, 0.4 , 0.2, 0.1, 0.05, 0.01]
SEQUENCE_LENGTH = 1000

## Imports

In [2]:
import sys
import os
import numpy as np

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from src.mutations import mutate_seq, random_base

In [3]:
!pip install numba

## Base sequences

In [4]:
base_sequence = []
for j in range(SEQUENCE_LENGTH):
    base_sequence.append(random_base())

mutated_sequences = []
for i in MUTATIONS:
    amount_mutations = int(SEQUENCE_LENGTH * i)
    mutated_sequences.append(list(mutate_seq(base_sequence, amount_mutations)[0]))

shortened_sequences = []
for j in REMOVALS:
    amount_removals = int(SEQUENCE_LENGTH * j)
    # We remove random positions from the base sequence
    shortened_seq = base_sequence.copy()
    for k in range(amount_removals):
        pos_to_remove = np.random.randint(0, len(shortened_seq))
        shortened_seq.pop(pos_to_remove)
    shortened_sequences.append(shortened_seq)



In [5]:
print(type(base_sequence))
print(type(mutated_sequences))
print(type(mutated_sequences[0]))
print(type(shortened_sequences))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


## Import algorithims

In [6]:
from tests.algorithms_all import needleman_wunsch, needleman_wunsch_diag_numpy, needleman_wunsch_numba, needleman_wunsch_diagbounded_numpy

In [7]:
algorithims = [
    needleman_wunsch,
    needleman_wunsch_diag_numpy,
    needleman_wunsch_numba,
    needleman_wunsch_diagbounded_numpy
]

results = {}

for algorithm in algorithims:
    print(f"Benchmarking {algorithm.__name__} with mutations:")
    for seq in mutated_sequences:
        score = algorithm(base_sequence, seq)
        print(f"Score: {score[0]}")
        results.setdefault(algorithm.__name__, []).append(score[0])
    
    print(f"Benchmarking {algorithm.__name__} with removals:")
    for seq in shortened_sequences:
        score = algorithm(base_sequence, seq)
        print(f"Score: {score[0]}")
        results.setdefault(algorithm.__name__, []).append(score[0])

Benchmarking needleman_wunsch with mutations:
Score: -7152
Score: -7122
Score: -7090
Score: -7052
Score: -7034
Score: -7012
Score: -6994
Benchmarking needleman_wunsch with removals:
Score: -1400
Score: -800
Score: -200
Score: 400
Score: 700
Score: 850
Score: 970
Benchmarking needleman_wunsch_diag_numpy with mutations:
Score: -7152.0
Score: -7122.0
Score: -7090.0
Score: -7052.0
Score: -7034.0
Score: -7012.0
Score: -6994.0
Benchmarking needleman_wunsch_diag_numpy with removals:
Score: -1400.0
Score: -800.0
Score: -200.0
Score: 400.0
Score: 700.0
Score: 850.0
Score: 970.0
Benchmarking needleman_wunsch_numba with mutations:
Score: -7152
Score: -7122
Score: -7090
Score: -7052
Score: -7034
Score: -7012
Score: -6994
Benchmarking needleman_wunsch_numba with removals:
Score: -1400
Score: -800
Score: -200
Score: 400
Score: 700
Score: 850
Score: 970
Benchmarking needleman_wunsch_diagbounded_numpy with mutations:
Score: -7658.0
Score: -7662.0
Score: -7674.0
Score: -7676.0
Score: -7690.0
Score: -76

## Compare results

In [8]:
!pip install pandas

In [9]:
import pandas as pd

# Recreate DataFrame from results (ensures consistent ordering)
df = pd.DataFrame.from_dict(results, orient='index')

# Build column names: first len(MUTATIONS) are mutation cases, next len(REMOVALS) are removal cases
mut_cols = [f"mut-{v}" for v in MUTATIONS]
rem_cols = [f"rem-{v}" for v in REMOVALS]
new_cols = mut_cols + rem_cols

# Handle potential length mismatch defensively
if len(new_cols) != df.shape[1]:
    if len(new_cols) > df.shape[1]:
        new_cols = new_cols[: df.shape[1]]
    else:
        extra = [f"case_{i+1}" for i in range(len(new_cols), df.shape[1])]
        new_cols = new_cols + extra

df.columns = new_cols

# Display the table
display(df.T)


,needleman_wunsch,needleman_wunsch_diag_numpy,needleman_wunsch_numba,needleman_wunsch_diagbounded_numpy
mut-0.8,-7152.0,-7152.0,-7152.0,-7658.0
mut-0.6,-7122.0,-7122.0,-7122.0,-7662.0
mut-0.4,-7090.0,-7090.0,-7090.0,-7674.0
mut-0.2,-7052.0,-7052.0,-7052.0,-7676.0
mut-0.1,-7034.0,-7034.0,-7034.0,-7690.0
mut-0.05,-7012.0,-7012.0,-7012.0,-7672.0
mut-0.01,-6994.0,-6994.0,-6994.0,-7662.0
rem-0.8,-1400.0,-1400.0,-1400.0,-1424.0
rem-0.6,-800.0,-800.0,-800.0,-832.0
rem-0.4,-200.0,-200.0,-200.0,-200.0
